## Section 1: Environment Setup

In [ ]:
# Check GPU
!nvidia-smi
!pwd

In [ ]:
# Clone repository
!git clone https://github.com/Japyh/llm-based-dbms.git
%cd llm-based-dbms
!ls -lh data/

In [ ]:
# Install dependencies
!pip install -q -r requirements.txt
!pip install -q transformers datasets accelerate bitsandbytes peft trl scikit-learn

import transformers
import peft
print(f"Transformers: {transformers.__version__}")
print(f"PEFT: {peft.__version__}")

In [ ]:
# Copy or link the fine-tuned adapter
# Option 1: If running right after training notebook, adapter is already in ./nl2sql-mistral-lora
# Option 2: If uploaded as Kaggle dataset, copy it here

import os
from pathlib import Path

adapter_dir = Path("nl2sql-mistral-lora")

# Check if adapter exists
if adapter_dir.exists():
    print(f"✓ Adapter found at {adapter_dir}")
    print(f"Contents: {list(adapter_dir.iterdir())}")
else:
    print(f"⚠ Adapter not found at {adapter_dir}")
    print("If you uploaded it as a Kaggle dataset, copy it here:")
    print("  !cp -r /kaggle/input/your-adapter-dataset/nl2sql-mistral-lora .")

## Section 2: Load Model and Adapter

In [ ]:
import sys
sys.path.append(".")
from src.config import BASE_HF_NL2SQL_MODEL

print(f"Base model: {BASE_HF_NL2SQL_MODEL}")
print(f"Adapter directory: {adapter_dir}")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(adapter_dir)
print(f"✓ Tokenizer loaded from adapter directory")

# For evaluation, we can use 4-bit to save memory (or 16-bit for faster inference)
# Using 4-bit here for consistency with training
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print(f"Loading base model {BASE_HF_NL2SQL_MODEL}...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_HF_NL2SQL_MODEL,
    quantization_config=bnb_config,
    device_map="auto"
)

print(f"Loading LoRA adapter from {adapter_dir}...")
model = PeftModel.from_pretrained(base_model, adapter_dir)
model.eval()

print("✓ Model loaded successfully!")

## Section 3: Load Test Dataset

In [ ]:
import json
from sklearn.model_selection import train_test_split

# Load the full dataset
dataset_path = Path("data/nl2sql_train_chat_raw.jsonl")
examples = []
with open(dataset_path, "r", encoding="utf-8") as f:
    for line in f:
        examples.append(json.loads(line))

print(f"Total examples: {len(examples)}")

# Use same split as training (20% validation)
_, test_examples = train_test_split(
    examples,
    test_size=0.2,
    random_state=42  # Same seed as training
)

print(f"Test examples: {len(test_examples)}")
print(f"\nFirst test example:")
print(json.dumps(test_examples[0], indent=2))

## Section 4: Evaluation Functions

In [ ]:
def generate_sql(question: str, system_prompt: str = None) -> str:
    """
    Generate SQL from a natural language question.
    """
    if system_prompt is None:
        system_prompt = (
            "You are a Text-to-SQL assistant for our SQLite sales database. "
            "Return only a valid SQL SELECT query, with no explanation, no comments, "
            "and no natural language. Never modify data or schema."
        )
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]
    
    # Apply chat template
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    # Tokenize
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.1,
            do_sample=True,
            top_p=0.95,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    # Decode
    response = tokenizer.decode(
        outputs[0][inputs.input_ids.shape[1]:],
        skip_special_tokens=True
    )
    
    return response.strip()

print("✓ Generation function ready")

In [ ]:
from src.db.core import run_query

def is_sql_executable(sql: str) -> bool:
    """
    Check if SQL query executes without error.
    """
    try:
        run_query(sql)
        return True
    except Exception:
        return False

def normalize_sql(sql: str) -> str:
    """
    Normalize SQL for comparison (lowercase, strip whitespace).
    """
    return " ".join(sql.lower().split())

print("✓ Evaluation utilities ready")

## Section 5: Run Evaluation

In [ ]:
# Evaluate on all test examples
results = []

print(f"Evaluating on {len(test_examples)} test examples...\n")

for i, example in enumerate(test_examples):
    messages = example["messages"]
    system_msg = messages[0]["content"]
    user_msg = messages[1]["content"]
    reference_sql = messages[2]["content"]
    
    # Generate SQL
    generated_sql = generate_sql(user_msg, system_msg)
    
    # Evaluate
    is_executable = is_sql_executable(generated_sql)
    is_exact_match = normalize_sql(generated_sql) == normalize_sql(reference_sql)
    
    result = {
        "question": user_msg,
        "reference_sql": reference_sql,
        "generated_sql": generated_sql,
        "is_executable": is_executable,
        "is_exact_match": is_exact_match
    }
    results.append(result)
    
    # Print progress
    if (i + 1) % 10 == 0:
        print(f"Processed {i + 1}/{len(test_examples)} examples...")

print(f"\n✓ Evaluation complete!")

## Section 6: Compute Metrics

In [ ]:
# Calculate metrics
total = len(results)
executable_count = sum(1 for r in results if r["is_executable"])
exact_match_count = sum(1 for r in results if r["is_exact_match"])

executable_rate = (executable_count / total) * 100
exact_match_rate = (exact_match_count / total) * 100

metrics = {
    "total_examples": total,
    "executable_count": executable_count,
    "executable_rate": executable_rate,
    "exact_match_count": exact_match_count,
    "exact_match_rate": exact_match_rate
}

print("=" * 70)
print("EVALUATION METRICS")
print("=" * 70)
print(f"Total test examples:        {total}")
print(f"Executable SQL queries:     {executable_count} ({executable_rate:.1f}%)")
print(f"Exact match SQL:            {exact_match_count} ({exact_match_rate:.1f}%)")
print("=" * 70)

## Section 7: Show Example Predictions

In [ ]:
import random

# Show some examples
print("\n" + "=" * 70)
print("SAMPLE PREDICTIONS")
print("=" * 70)

# Show a few random examples
sample_results = random.sample(results, min(5, len(results)))

for i, result in enumerate(sample_results, 1):
    print(f"\nExample {i}:")
    print(f"Question: {result['question']}")
    print(f"Reference SQL: {result['reference_sql']}")
    print(f"Generated SQL: {result['generated_sql']}")
    print(f"Executable: {result['is_executable']} | Exact Match: {result['is_exact_match']}")
    print("-" * 70)

In [ ]:
# Show failures (if any)
failures = [r for r in results if not r["is_executable"]]

if failures:
    print(f"\n⚠ {len(failures)} FAILED QUERIES (non-executable):")
    print("=" * 70)
    
    for i, result in enumerate(failures[:5], 1):  # Show first 5
        print(f"\nFailure {i}:")
        print(f"Question: {result['question']}")
        print(f"Generated SQL: {result['generated_sql']}")
        print("-" * 70)
else:
    print("\n✓ All generated queries are executable!")

## Section 8: Save Evaluation Results

In [ ]:
# Save metrics to JSON
output_file = Path("nl2sql_eval_metrics.json")

eval_output = {
    "metrics": metrics,
    "sample_results": results[:10],  # Save first 10 for inspection
    "failures": failures[:10] if failures else []
}

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(eval_output, f, indent=2, ensure_ascii=False)

print(f"✓ Evaluation results saved to {output_file}")
print(f"\nYou can download this file from Kaggle's Output section.")

## Summary

### Next Steps

1. **Download evaluation metrics**: Get `nl2sql_eval_metrics.json` from Output
2. **Document results**: Add metrics to your README or project documentation
3. **If results are good**:
   - Deploy the adapter to `models/nl2sql-mistral-lora/` in your repo
   - Implement full `LocalHFLLMProvider` in `src/llm/provider.py`
   - Integrate with `NL2SQLEngine`
4. **If results need improvement**:
   - Increase training epochs
   - Expand dataset with more examples
   - Try different LoRA hyperparameters
   - Experiment with different base models

### Typical Results to Expect

For a well-fine-tuned model on 220 examples:
- **Executable rate**: 85-95% (most queries run without syntax errors)
- **Exact match rate**: 30-60% (exact string match is strict; semantic equivalence is higher)

Even if exact match is lower, queries that are executable and produce correct results are still valuable!